# Keras implementation for training CNN with kaggle MNIST dataset

In [1]:
#mount driver with data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#read data
import pandas as pd
import numpy as np
import tensorflow as tf
import os

train_set = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/MNIST_CNN/data/train.csv")
test_set = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/MNIST_CNN/data/test.csv")

In [3]:
train_set.head(10)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#normalize and seperate labels from train/test set
labels = train_set.pop('label')
features = train_set.values / 255.0
test_features = test_set.values / 255.0
features = features.astype('float32')
test_features = test_features.astype('float32')
features = features.reshape(features.shape[0],28,28,1)
test_features = test_features.reshape(test_features.shape[0],28,28,1)
print(features.shape)
print(labels.shape)

(42000, 28, 28, 1)
(42000,)


In [5]:
#checking labels
print(labels[0:10])

0    1
1    0
2    1
3    4
4    0
5    0
6    7
7    3
8    5
9    3
Name: label, dtype: int64


In [0]:
#use get_dummies for one-hot encoding and convert to numpy array
labels = pd.get_dummies(labels)
labels = labels.values

In [7]:
print(labels[0:10])

[[0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]]


In [8]:
#create model
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Conv2D(40, kernel_size=5, padding="same",input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(70, kernel_size=3, padding="same"))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(500, kernel_size=3, padding="same"))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(1024, kernel_size=3, padding="valid"))
model.add(BatchNormalization())
model.add(LeakyReLU())

#Flatten and use fully-connected layers to output labels
model.add(Flatten())
model.add(Dense(units=100))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.1))
model.add(Dense(units=100))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.1))
model.add(Dense(units=100))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.3))
model.add(Dense(10))
model.add(Activation("softmax"))

#compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


In [13]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
kf = KFold(n_splits=5,shuffle=True)
for train_index, test_index in kf.split(features,labels):
  X_train, y_train = features[train_index], labels[train_index]
  X_test, y_test = features[test_index], labels[test_index]
  model.fit(X_train,y_train,epochs=32,shuffle=True,verbose=1,validation_data=(X_test, y_test))

Train on 33600 samples, validate on 8400 samples
Epoch 1/32
33600/33600 [==============================] - 92s 3ms/step - loss: 0.0307 - acc: 0.9910 - val_loss: 0.0193 - val_acc: 0.9945
Epoch 2/32
33600/33600 [==============================] - 86s 3ms/step - loss: 0.0207 - acc: 0.9938 - val_loss: 0.0162 - val_acc: 0.9958
Epoch 3/32
33600/33600 [==============================] - 85s 3ms/step - loss: 0.0167 - acc: 0.9953 - val_loss: 0.0200 - val_acc: 0.9948
Epoch 4/32
33600/33600 [==============================] - 87s 3ms/step - loss: 0.0137 - acc: 0.9956 - val_loss: 0.0244 - val_acc: 0.9937
Epoch 5/32
33600/33600 [==============================] - 87s 3ms/step - loss: 0.0126 - acc: 0.9962 - val_loss: 0.0300 - val_acc: 0.9913
Epoch 6/32
33600/33600 [==============================] - 90s 3ms/step - loss: 0.0106 - acc: 0.9965 - val_loss: 0.0295 - val_acc: 0.9924
Epoch 7/32
33600/33600 [==============================] - 90s 3ms/step - loss: 0.0097 - acc: 0.9967 - val_loss: 0.0220 - val_acc:

In [0]:
#save model
model.save("/content/gdrive/My Drive/Colab Notebooks/MNIST_CNN/model/model1.h5")

In [15]:
#predict labels
predicted_labels = model.predict_classes(test_features)
print(predicted_labels)

[2 0 9 ... 3 9 2]


In [0]:
#make a submit csv file for kaggle submission
submit = pd.DataFrame({"ImageId" : range(1,len(predicted_labels) + 1),
                       "Label" : predicted_labels})
submit.to_csv("/content/gdrive/My Drive/Colab Notebooks/MNIST_CNN/submit.csv", index = False)